In [1]:
import pandas as pd
import requests
import urllib.parse
import bs4
import re, os, hashlib, datetime
from json_memoize import memoize

q = urllib.parse.quote_plus

# requirements: multiprocesspandas
#from multiprocesspandas import applyparallel # unused but required
#from pandas_parallel_apply import DataFrameParallel, apply_on_df_parallel # pip install pandas-parellel-apply, pathos


# search page: https://services.wakegov.com/realestate/Search.asp

cache_dir = "cache_dir/"
if not os.path.isdir(cache_dir):
    os.mkdir(cache_dir)

def call_api(url: str) -> str:
    # get result, store in cache
    req = requests.get(url)
    req.raise_for_status()

    hash_fname = hashlib.md5(url.encode('utf-8')).hexdigest()[:30] + '.txt'
    path = os.path.join(cache_dir, hash_fname)
    if os.path.exists(path) and os.path.getsize(path) > 10:
        #print("cache hit")
        with open(path, 'r') as fp:
            return fp.read()
    else:
        #print("cache miss")
        result = req.text
        with open(path, 'w') as fp:
            fp.write(result)
        return req.text


df = pd.read_csv('nc_wake_addrlist.csv')
print(f"Number of potential properties: {len(df):,}")
#df = df.head(10) # DEBUG
df = df.sample(5000, random_state=1230984)
#df

Number of potential properties: 480,191


In [2]:
df['street_type'] = df.street.apply(lambda x: x.strip().split(' ')[-1])
df['street_short'] = df.street.apply(lambda x: ' '.join(x.strip().split(' ')[:-1]))
df[['street_type', 'number']].groupby('street_type').count().sort_values('number', ascending=False).head(20).T
#df

street_type,DR,RD,CT,LN,ST,WAY,PL,AVE,CIR,TRL,BLVD,LOOP,PKWY,CRSG,HWY,RUN,PT,SQ,PATH,WYND
number,1575,737,618,595,414,288,184,162,158,84,47,23,13,12,9,8,8,7,6,4


In [3]:
# PRE-WORK for do_search_for_street()
html = call_api('https://services.wakegov.com/realestate/ValidateAddress.asp?stnum=7634&stname=OLD+US+1&locidList=&spg=')
df1 = pd.read_html(html)
df1 = df1[4]
# promote headers
df1.columns = df1.iloc[0]
df1 = df1[1:-1] # remove header and hidden silly footer row

df1

,Line,Account,St Num,St Misc,Pfx,Street Name,Type,Sfx,ETJ,Owner
1,1,0022431,7634,NaN,NaN,OLD US 1,HWY,NaN,WC,DOMTAR PAPER CO LLC


In [4]:
@memoize(max_age=3600*24*7*2, cache_folder_path='./memo_cache_1')
def do_search_for_street(street_short: str, number: str) -> dict:
    html = call_api(f'https://services.wakegov.com/realestate/ValidateAddress.asp?stnum={q(number)}&stname={q(street_short)}&locidList=&spg=')
    out = {'acctlist_status': 'normal'}

    if 'Please refine your search by selecting one or more' in html:
        out['acctlist_status'] = 'error: wants a refined search'
        return out

    df1 = pd.read_html(html)
    df1 = df1[4]
    # promote headers
    df1.columns = df1.iloc[0]
    df1 = df1[1:-1] # remove header and hidden silly footer row

    df1 = df1.rename(columns=lambda x: 'acctlist_' + x)

    out['acctlist_row_count'] = len(df1)

    if len(df1) == 0:
        out['acctlist_status'] = 'error: no account results'
        return out
    
    if len(df1) > 1:
        out['acctlist_status'] = 'warning: >1 account result; using first one'

    out.update(df1.iloc[0].to_dict())

    return out

# do test
[
    do_search_for_street('A', '2301'),
    do_search_for_street('COOPER', '5201'),
]
    

[{'acctlist_status': 'normal',
  'acctlist_row_count': 1,
  'acctlist_Line': '1',
  'acctlist_Account': '0034575',
  'acctlist_St Num': '2301',
  'acctlist_St Misc': nan,
  'acctlist_Pfx': nan,
  'acctlist_Street Name': 'A',
  'acctlist_Type': 'ST',
  'acctlist_Sfx': nan,
  'acctlist_ETJ': 'RA',
  'acctlist_Owner': 'LAVANI, ARVIND S & VASANT A'},
 {'acctlist_status': 'normal',
  'acctlist_row_count': 1,
  'acctlist_Line': '1',
  'acctlist_Account': '0014316',
  'acctlist_St Num': '5201',
  'acctlist_St Misc': nan,
  'acctlist_Pfx': nan,
  'acctlist_Street Name': 'COOPER',
  'acctlist_Type': 'RD',
  'acctlist_Sfx': nan,
  'acctlist_ETJ': 'WC',
  'acctlist_Owner': 'MACLACHLAN, BRUCE H'}]

In [5]:
# call in prod: do_search_for_street()
def handle_call(func, *args):
    try:
        return func(*args)
    except:
        print(f"Error with call args: {args}")
        return {}

# df['acctlist'] = df.apply_parallel(lambda r, static_data: static_data[0](static_data[1], str(r.get('street_short', '')), str(r.get('number', ''))), axis=1, num_processes=10, static_data=[handle_call, do_search_for_street])
#df['acctlist'] = apply_on_df_parallel(df, lambda r: do_search_for_street(str(r.street_short), str(r.number)), n_cores=5, pbar=True)

df['acctlist'] = df.apply(lambda r: handle_call(do_search_for_street, str(r.street_short), str(r.number)), axis=1)
#df = pd.concat([df, pd.json_normalize(df['acctlist'])], axis=1)
df = pd.concat([df, df['acctlist'].apply(pd.Series)], axis=1)

# filter out null Accounts (which are caused by a search page being loaded, generally)
orig_len = len(df)
df = df[pd.notna(df['acctlist_Account'])]

print(f"{len(df)}/{orig_len} addresses retrieved an Account value.")
df

Error with call args: ('W ELM', '304')
Error with call args: ('S WEST', '523')
Error with call args: ('N MAIN', '430')
Error with call args: ('N COALPORT', '605')
Error with call args: ('N SELMA', '46')
Error with call args: ('S BEAVER', '3420')
Error with call args: ('E MARTIN', '703')
Error with call args: ('E MILLBROOK', '624')
Error with call args: ('S SAUNDERS', '2539')
Error with call args: ('N FRANKLIN', '832')
Error with call args: ('N CHURCH', '913')
Error with call args: ('W JOHNSON', '809')
Error with call args: ('S WHITE', '527')
Error with call args: ('N MAIN', '20')
Error with call args: ('N DRAWBRIDGE', '104')
Error with call args: ('N WHITE', '103')
Error with call args: ('W VANCE', '203')
Error with call args: ('E DAVIE', '549')
Error with call args: ('W CORNWALL', '245')
Error with call args: ('N TAYLOR', '901')
Error with call args: ('N NEW HOPE', '4721')
Error with call args: ('E HONEY SPRINGS', '1721')
Error with call args: ('E CORNWALL', '411')
Error with call arg

: 

In [ ]:
# PRE-WORK for do_lookup_accountnum()
html = call_api(f'https://services.wakegov.com/realestate/Account.asp?id=0118000')
out = {'acctsearch_status': 'normal'}
html_txt = re.sub('<[^<]+?>', ' ', html)
html_txt = re.sub(r'(&nbsp;)+', ' ', html_txt)
html_txt = re.sub(r'(\s)+', ' ', html_txt)
#print(html_txt)
out['acctsearch_Real Estate ID'] = re.search(r"Real Estate ID (\d{5,15})", html_txt).group(1)
out['acctsearch_PIN'] = re.search(r"PIN # (\d{5,15})", html_txt).group(1)
out['acctsearch_Property Description'] = re.search(r"Property Description (.+) Pin/Parcel History New Search", html_txt).group(1).strip()
out['acctsearch_Property Owner'] = re.search(r"Property Owner (.+) \(Use the Deeds link to", html_txt).group(1).strip()
out['acctsearch_Property Location Address'] = re.search(r"Property Location Address (.+) Administrative Data Old Map", html_txt).group(1).strip()

soup = bs4.BeautifulSoup(html)
for tr in soup.find_all('tr'):
    td_childs = tr.find_all('td')
    if len(td_childs) == 2:
        out['acctsearchf_' + td_childs[0].text.strip()] = td_childs[1].text.strip()

print(out)


{'acctsearch_status': 'normal', 'acctsearch_Real Estate ID': '0118000', 'acctsearch_PIN': '1706874307', 'acctsearch_Property Description': '308 ROSEHAVEN DR LO96 COLONY WOODS SE2 BM1980-815', 'acctsearch_Property Owner': 'WILLIAMS, JAMES C III', 'acctsearch_Property Location Address': '308 ROSEHAVEN DR RALEIGH NC 27609-3882', 'acctsearchf_Account Summary': 'Tax Bills', 'acctsearchf_Real Estate ID \n\n\t\t\t\t\t\t\t\t\t0118000': 'PIN #\xa0\xa01706874307', 'acctsearchf_': '&nbsp', 'acctsearchf_Location Address': 'Property Description', 'acctsearchf_308  ROSEHAVEN DR': 'LO96 COLONY WOODS SE2 BM1980-815', 'acctsearchf_Administrative Data': '', 'acctsearchf_Old Map #': '380-00000-1199', 'acctsearchf_Map/Scale': '1706 12', 'acctsearchf_VCS': '07RA006', 'acctsearchf_City': 'RALEIGH', 'acctsearchf_Fire District': '', 'acctsearchf_Township': 'HOUSE CREEK', 'acctsearchf_Land Class': 'R-<10-HS', 'acctsearchf_ETJ': 'RA', 'acctsearchf_Spec Dist(s)': '', 'acctsearchf_Zoning': 'R-4', 'acctsearchf_His

In [ ]:
@memoize(max_age=3600*24*7*2, cache_folder_path='./memo_cache_2')
def do_lookup_accountnum(account_num: str) -> dict:
    #print(f"Doing Account: '{account_num}'")
    out = {'acctsearch_status': 'normal'}

    if pd.isna(account_num):
        out['acctsearch_status'] = 'error: no account number'
        return out

    html = call_api(f'https://services.wakegov.com/realestate/Account.asp?id={account_num}')
    html = re.sub(r'(&nbsp;?)+', ' ', html)
    html = re.sub(r'(\s)+', ' ', html)


    html_txt = re.sub('<[^<]+?>', ' ', html)
    html_txt = re.sub(r'(\s)+', ' ', html_txt)
    
    #print(html_txt)
    try:
        out['acctsearch_Real Estate ID'] = re.search(r"Real Estate ID (\d{5,15})", html_txt).group(1)
    except:
        print(f"Unknown Real Estate ID on Account '{account_num}'")
    out['acctsearch_PIN'] = re.search(r"PIN # (\d{5,15})", html_txt).group(1)
    out['acctsearch_Property Description'] = re.search(r"Property Description (.+) Pin/Parcel History New Search", html_txt).group(1).strip()
    out['acctsearch_Property Owner'] = re.search(r"Property Owner (.+) \(Use the Deeds link to", html_txt).group(1).strip()
    out['acctsearch_Property Location Address'] = re.search(r"Property Location Address (.+) Administrative Data Old Map", html_txt).group(1).strip()

    soup = bs4.BeautifulSoup(html)
    for tr in soup.find_all('tr'):
        td_childs = tr.find_all('td')
        if len(td_childs) == 2:
            field_name = td_childs[0].text.strip()
            if len(field_name) < 3:
                continue
            if field_name[:10] == out['acctsearch_Property Description'][:10]:
                continue
            if field_name.startswith('Real Estate ID'):
                continue
            out['acctsearchf_' + field_name] = td_childs[1].text.strip() # acctsearchf, f stands for field

    return out

[do_lookup_accountnum('0118000').keys(), do_lookup_accountnum('0130902')]

[dict_keys(['acctsearch_status', 'acctsearch_Real Estate ID', 'acctsearch_PIN', 'acctsearch_Property Description', 'acctsearch_Property Owner', 'acctsearch_Property Location Address', 'acctsearchf_Account Summary', 'acctsearchf_Location Address', 'acctsearchf_Administrative Data', 'acctsearchf_Old Map #', 'acctsearchf_Map/Scale', 'acctsearchf_VCS', 'acctsearchf_City', 'acctsearchf_Fire District', 'acctsearchf_Township', 'acctsearchf_Land Class', 'acctsearchf_ETJ', 'acctsearchf_Spec Dist(s)', 'acctsearchf_Zoning', 'acctsearchf_History ID 1', 'acctsearchf_History ID 2', 'acctsearchf_Acreage', 'acctsearchf_Permit Date', 'acctsearchf_Permit #', 'acctsearchf_Transfer Information', 'acctsearchf_Deed Date', 'acctsearchf_Book & Page', 'acctsearchf_Revenue Stamps', 'acctsearchf_Pkg Sale Date', 'acctsearchf_Pkg Sale Price', 'acctsearchf_Land Sale Date', 'acctsearchf_Land Sale Price', 'acctsearchf_Total Units', 'acctsearchf_Recycle Units', 'acctsearchf_Apt/SC Sqft', 'acctsearchf_Heated Area', 'ac

In [ ]:
# call in prod: do_lookup_accountnum()

df['acctsearch'] = df.apply(lambda r: do_lookup_accountnum(str(r['acctlist_Account'])), axis=1)
#df = pd.concat([df, pd.json_normalize(df['acctsearch'])], axis=1)
df = pd.concat([df, df['acctsearch'].apply(pd.Series)], axis=1)


In [ ]:
# add in additional cols

def move_cols_to_end(df: pd.DataFrame, col_list: list) -> pd.DataFrame:
    for col in col_list:
        df.insert(len(df.columns)-1, col, df.pop(col))
    return df

def ree(pattern: str, string: str, group=1) -> str:
    try:
        return re.search(pattern, string).group(group)
    except:
        return None


df['state'] = 'NC' # this must be the first column added
df['property_zip5'] = df['acctsearch_Property Location Address'].apply(lambda v: ree(r"(\d{5})-\d+", v))
df['property_street_address'] = df.apply(lambda r: f"{r['acctlist_St Num']} {r['acctlist_Street Name']} {r['acctlist_Type']}", axis=1)
df['property_county'] = 'WAKE' # it's all the same one
df['property_id'] = df.apply(lambda r: f"{r['acctsearch_PIN']}-{r['acctlist_Account']}", axis=1) # PIN and Account Number (TODO zero-pad Account maybe)
df['building_year_built'] = pd.to_datetime(df['acctsearchf_Permit Date']).dt.year.astype(str).str.replace(r'(\.0)|(nan)','', regex=True, case=False)
df['source_url'] = df['acctlist_Account'].apply(lambda v: f'https://services.wakegov.com/realestate/Account.asp?id={v}')
df['book'] = df['acctsearchf_Book & Page'].apply(lambda v: v.split(' ')[0] if v.split(' ')[0].isnumeric() else None)
df['page'] = df['acctsearchf_Book & Page'].apply(lambda v: ree(r"\s(.+)", v))
df['deed_date'] = pd.to_datetime(df['acctsearchf_Deed Date'])

df['sale_price'] = df.apply(lambda r: r['acctsearchf_Pkg Sale Price'] if pd.notna(r['acctsearchf_Pkg Sale Price']) else r['acctsearchf_Land Sale Price'], axis=1)
df['sale_datetime'] = pd.to_datetime(df.apply(lambda r: r['acctsearchf_Pkg Sale Date'] if pd.notna(r['acctsearchf_Pkg Sale Price']) else r['acctsearchf_Land Sale Date'], axis=1))

df['sale_datetime'] = df.apply(lambda r: r['deed_date'] if pd.isna(r['sale_datetime']) else r['sale_datetime'], axis=1)

# remove null sale_datetime rows as it's mandatory
len_before = len(df)
df = df[pd.notna(df['sale_datetime'])]

print(f"Kept {len(df)}/{len_before} rows by final filters (like sale_datetime).")

# TODO building stories/beds from the buildings tab
df['land_area_acres'] = df['acctsearchf_Acreage'].astype(float)
df['land_area_sqft'] = df['land_area_acres'].apply(lambda v: v*43560)

rename_list = {
    "acctsearchf_Land Class": "property_type",
    "acctsearch_Property Owner": "buyer_1_name",
    "acctsearchf_Township": "property_township",
    "latitude": "property_lat",
    "longitude": "property_lon",
    "acctsearchf_Heated Area": "building_area_sqft",
    "acctsearchf_Bldg. Value Assessed": "building_assessed_value",
    "acctsearchf_Land Value Assessed": "land_assessed_value",

    "acctsearchf_City": "property_city",
}
df = df.rename(columns=rename_list)
df = move_cols_to_end(df, rename_list.values())

for col in ['sale_price', 'building_assessed_value', 'land_assessed_value', 'building_area_sqft']:
    df[col] = df[col].str.replace(r"[*,$]", '', regex=True).astype(float, errors='ignore')

df['sale_price'] = df['sale_price'].str.replace('^$', '0', regex=True).astype(float, errors='ignore').astype(int)

# delete cols to the left of 'state'
new_cols = []
hit_first_col_yet = False
for col in df.columns:
    if col == 'state': hit_first_col_yet = True
    if hit_first_col_yet: new_cols.append(col)
df_out = df[new_cols]

df_out.T
df_out.to_csv('out_' + datetime.datetime.now().strftime('%Y-%m-%d-%H%M%S') + '.csv', index=False)


Kept 466/466 rows by final filters (like sale_datetime).
